In [1]:
from src.api.factor import FactorAPI
app = FactorAPI.Test.FactorPerf(end_dt = 20240131)

Basic module imported!
INSTANCE_RECORD can be accessed to check trainer , account or else...
Getting random factor values...
Running factor test for 2 factors in BM [defaults], write_down=True, display_figs=False
FactorPerfManager calc Finished!
FactorPerfManager plot Finished!
Analytic Test of Factor Performance datas are saved to D:\Coding\learndl\learndl\results\test\perf\random_factor\data.xlsx
Analytic Test of Factor Performance plots are saved to D:\Coding\learndl\learndl\results\test\perf\random_factor\plot.pdf


In [2]:
from src.api.factor import FactorAPI
app = FactorAPI.Test.FmpOptim(start_dt=20240101 , end_dt = 20240131 , prob_type = 'linprog')


Getting random factor values...
Running optim test for 2 factors in BM [defaults], write_down=True, display_figs=False
Optimization of PortfolioBuilderGroup(2 alphas , 3 benchmarks , 2 lags , 1 param_groups , 5 dates , (60 builds) start!
Optimization of    0th [Fmp.factor1.csi300.Optim.lag0 ] Finished at 20240102 , time cost (ms) : {'parse_input': 415.23, 'solve': 138.93, 'output': 11.02}
Optimization of   50th [Fmp.factor1.csi1000.Optim.lag0] Finished at 20240130 , time cost (ms) : {'parse_input': 33.35, 'solve': 225.04, 'output': 24.92}
Group Optimization Finished , Total time: 15.32 secs, each optim time: 0.26
Group Accounting Finished , Total time: 2.93 secs.
FmpOptimManager calc Finished!
FmpOptimManager plot Finished!
Analytic Test of Optim Portfolio datas are saved to D:\Coding\learndl\learndl\results\test\optim\random_factor\data.xlsx
Analytic Test of Optim Portfolio plots are saved to D:\Coding\learndl\learndl\results\test\optim\random_factor\plot.pdf


In [3]:
from src.api.factor import FactorAPI
app = FactorAPI.Test.FmpTop(end_dt = 20240131)

Getting random factor values...
Running top test for 2 factors in BM [defaults], write_down=True, display_figs=False
Generation of PortfolioBuilderGroup(2 alphas , 3 benchmarks , 1 lags , 4 param_groups , 5 dates , (120 builds) start!
Generation of    0th [Fmp.factor1.csi300.Top 20.lag0 ] Finished at 20240102 , time cost (ms) : {'parse_input': 0.0, 'solve': 27.52, 'output': 18.0}
Generation of   50th [Fmp.factor1.csi300.Top 50.lag0 ] Finished at 20240116 , time cost (ms) : {'parse_input': 0.0, 'solve': 33.29, 'output': 16.62}
Generation of  100th [Fmp.factor1.csi500.Top 20.lag0 ] Finished at 20240130 , time cost (ms) : {'parse_input': 0.0, 'solve': 17.15, 'output': 30.39}
Group Generation Finished , Total time: 6.76 secs, each optim time: 0.06
Group Accounting Finished , Total time: 9.36 secs.
FmpTopManager calc Finished!
FmpTopManager plot Finished!
Analytic Test of Top Portfolio datas are saved to D:\Coding\learndl\learndl\results\test\top\random_factor\data.xlsx
Analytic Test of Top

In [1]:
import src
from src.basic import RegisteredModel
model = RegisteredModel.SelectModels()[0]
model.pred_dates , model.model_dates

Basic module imported!
INSTANCE_RECORD can be accessed to check trainer , account or else...


(array([20241104, 20241105, 20241106, 20241107, 20241108, 20241111,
        20241112, 20241113, 20241114, 20241115, 20241118, 20241119,
        20241120, 20241121, 20241122, 20241125]),
 array([20170103, 20170704, 20171226, 20180627, 20181220, 20190624,
        20191217, 20200617, 20201214, 20210615, 20211209, 20220613,
        20221206, 20230606, 20231201, 20240604]))

In [1]:
import itertools
import numpy as np
import pandas as pd
from contextlib import nullcontext

from typing import Any , Literal

from src.basic import PATH , CALENDAR , RegisteredModel , SILENT
from src.factor.util import Port , StockFactor , Benchmark , Portfolio
from src.factor.builder import PortfolioBuilder

class ModelPredFmp:
    STEP = 5
    SUB_FMP_TYPE = ['indep' , 'conti'] # independent and continuous portfolios

    def __init__(self , reg_model : RegisteredModel):
        self.reg_model = reg_model
        self.fmp_tables : dict[int , pd.DataFrame] = {}
    
    def pred_factor(self , date : int):
        df = self.reg_model.load_pred(date)
        assert not df.empty , f'empty df returned for {date}'
        factor = StockFactor(df.assign(date = date)).normalize()
        assert len(factor.factor_names) == 1 , f'expect 1 factor name , got {factor.factor_names}'
        return factor
    
    def alpha_model(self , date : int):
        return self.pred_factor(date).alpha_models()[0]

    def last_fmp_table(self , date : int):
        fmp_dates = CALENDAR.slice(self.reg_model.fmp_dates , None , CALENDAR.td(date , -self.STEP))
        if len(fmp_dates) == 0: return pd.DataFrame()
        last_date = fmp_dates.max()
        if last_date not in self.fmp_tables:
            self.fmp_tables[last_date] = self.reg_model.load_fmp(last_date)
        return self.fmp_tables[last_date]
    
    def last_port(self , date : int , port_name : str):
        table = self.last_fmp_table(date)
        if table.empty: return None
        table = table[table['name'] == port_name]
        return Portfolio.from_dataframe(table , name = port_name)

    def get_builder_kwargs(self , date : int , fmp_type : Literal['top' , 'optim'] , 
                           subtype : str , benchmark : str , n_best : int = -1):
        assert subtype in self.SUB_FMP_TYPE , f'invalid subtype: {subtype}'
        alpha = self.alpha_model(date)
        kwargs : dict[str , Any] = {
            'category' : fmp_type , 'alpha' : alpha , 'benchmark' : benchmark , 'lag' : 0 , 
            'suffixes' : [subtype] , 'n_best' : n_best , 'build_on' : None
        }
        if subtype == 'conti': kwargs['build_on'] = self.last_port(date , PortfolioBuilder.get_full_name(**kwargs))
        return kwargs
    
    def iter_builders(self , date : int , verbosity : int = 0):
        for subtype , benchmark , n_best in itertools.product(self.SUB_FMP_TYPE , Benchmark.DEFAULTS , [20 , 30 , 50 , 100]):
            kwargs = self.get_builder_kwargs(date , 'top' , subtype , benchmark , n_best)
            yield PortfolioBuilder(verbosity = verbosity , **kwargs)

        for subtype , benchmark in itertools.product(self.SUB_FMP_TYPE , Benchmark.DEFAULTS):
            kwargs = self.get_builder_kwargs(date , 'optim' , subtype , benchmark)
            yield PortfolioBuilder(verbosity = verbosity , **kwargs)
        
    def update_fmps(self , update = True , overwrite = False , silent = True):
        '''get update dates and build portfolios'''
        assert update != overwrite , 'update and overwrite must be different here'
        with SILENT if silent else nullcontext():   
            dates = CALENDAR.diffs(self.reg_model.pred_dates , self.reg_model.fmp_dates if update else [])
            self.build_fmps(dates , deploy = True)
        return dates
    
    def build_fmps(self , dates : list[int] | np.ndarray , deploy = True):
        for date in dates:
            self.fmp_tables[date] = self.build_day(date) 
            print(f'Finished build fmps for factor {self.reg_model.pred_name} at date {date}')
            if deploy: self.reg_model.save_fmp(self.fmp_tables[date] , date , False)
    
    def build_day(self , date : int):
        ports = [builder.setup().build(date).port.full_table for builder in fmp.iter_builders(date)]
        df = pd.concat(ports).reset_index(drop=True)
        assert df.columns.tolist() == ['name' , 'date' , 'secid' , 'weight'] , \
            f'expect columns: name , date , secid , weight , got {df.columns.tolist()}'
        return df
    

model = RegisteredModel.SelectModels()[0]
fmp = ModelPredFmp(model)
fmp.update_fmps()


Basic module imported!
INSTANCE_RECORD can be accessed to check trainer , account or else...


array([20241104, 20241105, 20241106, 20241107, 20241108, 20241111,
       20241112, 20241113, 20241114, 20241115, 20241118, 20241119,
       20241120, 20241121, 20241122, 20241125])

In [2]:
fmp.build_day(20241104)

,name,date,secid,weight
0,Top.gru_day_V0.csi300.Top_20.lag0.indep,20241104,651,0.050000
1,Top.gru_day_V0.csi300.Top_20.lag0.indep,20241104,800,0.050000
2,Top.gru_day_V0.csi300.Top_20.lag0.indep,20241104,688009,0.050000
3,Top.gru_day_V0.csi300.Top_20.lag0.indep,20241104,603993,0.050000
4,Top.gru_day_V0.csi300.Top_20.lag0.indep,20241104,601898,0.050000
...,...,...,...,...
2151,Optim.gru_day_V0.csi1000.default.lag0.conti,20241104,3029,0.000006
2152,Optim.gru_day_V0.csi1000.default.lag0.conti,20241104,603968,0.000004
2153,Optim.gru_day_V0.csi1000.default.lag0.conti,20241104,300609,0.000003
2154,Optim.gru_day_V0.csi1000.default.lag0.conti,20241104,601128,0.000002


In [2]:
fmp.reg_model.fmp_dates

array([], dtype=int32)

In [2]:
alpha = fmp.alpha_model(20241104)
alpha

AlphaModel (name=gru_day_V0)(1 days loaded)

In [3]:
from src.factor.util import *

In [5]:
builder = PortfolioBuilder('top' , alpha = alpha , benchmark = 'csi300' , build_on = None , verbosity  = 1 , n_best = 30)

In [6]:
builder.setup().build(20241104)

In initializing PortfolioGeneratorConfig, used kwargs: {'n_best': 30}


PortfolioBuilder(category='top',name='Fmp.gru_day_V0.csi300.Top_30.lag0',alpha='gru_day_V0',benchmark='csi300',lag=0,kwargs={'n_best': 30},1 fmp's,not accounted)

In [9]:
builder.portfolio.get(20241104).port

,secid,weight
0,301,0.033333
1,651,0.033333
2,688009,0.033333
3,603993,0.033333
4,601989,0.033333
5,601985,0.033333
6,601898,0.033333
7,601878,0.033333
8,601838,0.033333
9,601816,0.033333


In [2]:
fmp.build_day(20241104)

,name,date,secid,weight
0,a,20241103,159,0.058596
1,a,20241103,920002,0.056510
2,a,20241103,688085,0.053891
3,a,20241103,301319,0.052866
4,a,20241103,600617,0.052504
5,a,20241103,603810,0.048795
6,a,20241103,600477,0.047866
7,a,20241103,300706,0.047181
8,a,20241103,2846,0.046054
9,a,20241103,2256,0.044652
